In [23]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plp
import seaborn as sns

In [12]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Jahdrysql#09")

print(mydb)

In [20]:
mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")
for x in mycursor: 
    print(x)

('BD_Botiga',)
('information_schema',)
('mysql',)
('NBA',)
('performance_schema',)
('sys',)


In [19]:
mycursor.execute("CREATE DATABASE IF NOT EXISTS NBA")

In [22]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Jahdrysql#09",
    database ="NBA")

print(mydb)

In [ ]:
DB_NAME = 'NBA'

tables = {}
tables['teams'] = ("CREATE TABLE `teams` (League_ID ))